# Notes

This notebook is an example of a PRE-PROCESSING pipeline for satellite images with LAI data.
It is not meant to be run as a script, but rather to be used as a reference for how to use the preprocessing functions.

**IMPORTANT** This notebook assumes one has stored a series of *unpacked* RAS and RHD files in the VISTA format containing 
- The Leaf Area Index (LAI) values as a tensor of images over time.
- Information of the datetimes and the coordinates of the images. 

The VISTA format is not publicly available, but the functions in this notebook can be used as a reference for how to preprocess satellite images in general.

*Example scenario*: We have downloaded the RAS and RHD files containing the LAI values for a sentinel-2 tile (~12k by 12k image) over the span of 2020. We want to preprocess this data into timeseries for further analysis.

*Author*: Jens d'Hondt (TU Eindhoven)

In [3]:
# General imports
import os
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import sys
import seaborn as sns
from eolearn.core import EOPatch

# 0 Unpacking the data

This step unapacks a RAS-RHD file pair with LAI values into a series of .npy files, named by the date of the image.
(e.g. `2020_1_1.npy`, `2020_1_2.npy`, ...) 
The unpacking can be done using the `unpack_vista_unzipped` function in `stelar_spatiotemporal/preprocessing/vista_preprocessing.py`, as shown below.

**Assumptions**:
- One has downloaded and unzipped the RAS and RHD files to be processed.
- The RAS files can contain multiple images, each with a different date, month or year.

**Process**:
The function `unpack_vista_unzipped` will do the following:
1. Extract the images from the RAS file and store them in a series of .npy files named by the date of the individual image, based on the accompanied RHD file.

In [4]:
# Set minio credentials
os.environ["MINIO_ENDPOINT_URL"] = "http://localhost:9000"
os.environ["MINIO_ACCESS_KEY"] = "minioadmin"
os.environ["MINIO_SECRET_KEY"] = "minioadmin"

S3DATADIR = "s3://stelar-spatiotemporal/LAI"
LOCAL_DATADIR = "/tmp"

In [3]:
from sentinelhub import CRS
from stelar_spatiotemporal.preprocessing.vista_preprocessing import unpack_vista_unzipped

ras_path = os.path.join(S3DATADIR, "30TYQ_LAI_2020.RAS")
rhd_path = os.path.join(S3DATADIR, "30TYQ_LAI_2020.RHD")

outdir = os.path.join(LOCAL_DATADIR, "npys")

# Unpacks RAS and RHD files into numpy arrays
unpack_vista_unzipped(ras_path, rhd_path, outdir, crs=CRS(32630))

Unpacking 87 images from s3://stelar-spatiotemporal/LAI/30TYQ_LAI_2020.RAS


KeyboardInterrupt: 

# Option A: Batch processing the data

## 1 Combining images to eopatches

In this step we will combine the individual LAI images into a tensor, or package of images. 

**Assumptions**:
- One has stored the .npy files containing the images in a folder `$DATADIR` following the structure `DATADIR/npys/{date}.npy`.

**Process**:
The function `combine_bands` will do the following:
1. Partition the .npy files into a series of EOPatches, each containing a tensor of images, the corresponding datetimes and bounding box.

In [5]:
from stelar_spatiotemporal.preprocessing.preprocessing import combine_npys_into_eopatches, max_partition_size
from stelar_spatiotemporal.lib import load_bbox

npy_dir = os.path.join(LOCAL_DATADIR, "npys")

npy_paths = glob.glob(os.path.join(npy_dir,"*.npy"))
max_partition_size = max_partition_size(npy_paths[0], MAX_RAM=int(4 * 1e9))

bbox = load_bbox(os.path.join(npy_dir, "bbox.pkl"))

outpath = os.path.join(LOCAL_DATADIR, "lai_eopatch")
combine_npys_into_eopatches(npy_paths=npy_paths, 
                            outpath=outpath,
                            feature_name="LAI",
                            bbox=bbox,
                            partition_size=max_partition_size,
                            delete_after=True,
                            )

Processing 5 partitions of 19 dates each


# 2 LAI to CSV

In this step we will convert the LAI values to a timeseries of LAI values for each pixel OR field (extracted in the segmentation pipeline).
The conversion into timeseries will be done per image.

The values for collections of pixels and/or fields will be stored as column-major csv file with pixel/field ids as columns, and dates as rows. This is to facilitate appending of new data.
The values for pixels will be partitioned by *patchlets*, which are subsets of the full image. This is done to reduce the size of the csv files.

**Assumptions**:
- One has stored the EOPatch objects in a folder `$DATADIR` (see step 1).

## 2.1 LAI to CSV: pixel values

### 2.1.2 Approach 1: npys -> eopatches -> p patchlets per date -> p patchlets -> timeseries

**Process**:
The function `lai_to_csv_px` will do the following:

1. We break up each image into a series of patchlets.
2. We combine the data for each patchlet into a single eopatch.
3. We convert the eopatch into a timeseries of LAI values for each pixel.
4. (Optional) Combine the timeseries of LAI values for each pixel into a single csv file.

In [4]:
from stelar_spatiotemporal.preprocessing.timeseries import lai_to_csv_px

eop_dir = os.path.join(LOCAL_DATADIR, "lai_eopatch")
eop_paths = glob.glob(os.path.join(eop_dir, "partition_*"))
if len(eop_paths) == 0: eop_paths = [eop_dir]

patchlet_dir = os.path.join(LOCAL_DATADIR, "patchlets")

outdir = os.path.join(S3DATADIR, "lai_px_timeseries")

# Turn the LAI values into a csv file
lai_to_csv_px(eop_paths, patchlet_dir=patchlet_dir, outdir=outdir, delete_patchlets=False)

3. Extracting timeseries per patchlet:   5%|▍         | 4/81 [02:18<44:18, 34.52s/it]


KeyboardInterrupt: 

In [29]:
# Read the csv file
path = "/home/jens/ownCloud/Documents/3.Werk/0.TUe_Research/0.STELAR/VISTA/VISTA_workbench/data/pipeline_example/lai_px_timeseries/patchlet_4_4.csv"

df = pd.read_csv(path, index_col=0, parse_dates=True, usecols=np.arange(0, 4))
df[df < 0] = np.nan
df /= 1000
df.sort_index(inplace=True)
df.head(20)

,0_0,1_0,2_0
index,,,
2020-01-06,NaN,NaN,NaN
2020-01-11,3.049,3.430,4.242
2020-01-14,2.666,3.196,4.372
2020-01-16,3.192,3.408,4.193
2020-01-19,4.084,4.355,4.898
2020-02-03,1.891,2.129,3.051
2020-02-05,2.997,2.776,3.700
2020-02-15,1.063,1.309,1.589
2020-02-18,NaN,NaN,NaN


## 2.2 LAI to CSV: field values

**Process**:
The function `lai_to_csv_field` will do the following:

1. Temporarily save the (partitioned) eopatches as tiffs (necessary for masking with field shapes)
2. For each tiff:
    1. Load the LAI values.
    2. For each field:
        1. Mask the LAI values for the field.
        2. Take the median of the LAI values for the field for each date.
        3. Append the LAI values for the field to the corresponding csv file.
3. (Optional) Combine the timeseries of LAI values for each pixel into a single csv file.

In [5]:
from stelar_spatiotemporal.preprocessing.timeseries import lai_to_csv_field

eop_dir = os.path.join(LOCAL_DATADIR, "lai_eopatch")
eop_paths = glob.glob(os.path.join(eop_dir, "partition_*"))
if len(eop_paths) == 0: eop_paths = [eop_dir]

eop_paths.sort()
fields_path = "s3://stelar-spatiotemporal/fields.gpkg"

# Perform the process as described above
lai_to_csv_field(eop_paths, fields_path=fields_path, outdir=S3DATADIR, n_jobs=16, delete_tmp=False)

Processing eopatch 1/5


KeyboardInterrupt: 

In [9]:
# Read csv file
csv_path = os.path.join(DATADIR, "lai_field_timeseries.csv")
df_field = pd.read_csv(csv_path, index_col=0, usecols=np.arange(10))

df_field.head()

,0,3,4,5,6,7,8,9,10
2020-03-24,2104.0,542.0,387.5,271.0,171.0,3576.0,637.0,3268.0,2348.0
2020-01-06,NaN,NaN,NaN,807.0,NaN,2910.0,NaN,4304.0,NaN
2020-03-19,NaN,NaN,763.5,554.0,NaN,NaN,NaN,2084.0,NaN
2020-01-11,1754.0,1579.0,1117.0,869.0,267.5,2756.0,784.0,3920.0,2500.0
2020-01-14,2036.0,1118.0,922.0,786.0,204.0,1794.0,775.0,2728.0,1779.0
